In [1]:
from datasets import load_dataset
ds = load_dataset("brimmann2/squad-v2-sampled")

In [2]:
import torch
model_name = "google/gemma-3-1b-it"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [3]:
from transformers import AutoConfig, AutoTokenizer
from models.modeling_xgemma import XGemmaForCausalLM, XGemmaConfig

tokenizer = AutoTokenizer.from_pretrained(model_name)
xrag_token = "<xRAG>"
tokenizer.add_special_tokens({"additional_special_tokens": [xrag_token]})
xrag_token_id = tokenizer.convert_tokens_to_ids(xrag_token)

/home/brimmann/works/llm-recipes/models/checkpoint_handler.py:7: DeprecationWarning: `torch.distributed._shard.checkpoint` will be deprecated, use `torch.distributed.checkpoint` instead
  import torch.distributed._shard.checkpoint as dist_cp


In [4]:
retriever_hidden_size = 4096
config = XGemmaConfig.from_pretrained(
    model_name,
    projector_type='mlp2x_gelu',
    retriever_hidden_size=retriever_hidden_size,
)

model = XGemmaForCausalLM.from_pretrained(model_name, config=config)

# Resize token embeddings layer to account for the new special token
# model.resize_token_embeddings(len(tokenizer))  # I'm not if we need this

model.set_xrag_token_id(xrag_token_id)
model.to(device)
model.eval()

AttributeError: 'XGemmaConfig' object has no attribute 'layer_types'